Conceptos: tablas de historial, columnas de timestamp, soft delete, hash de fila para detectar cambios.

In [ ]:
-- Ejemplo conceptual: añadir columna updated_at
-- ALTER TABLE dbo.dim_clientes ADD updated_at DATETIME2 NULL;

In [ ]:
-- Hash para detectar cambios (MD5/Checksum) - demostración conceptual
SELECT cliente_id, nombre, segmento,
       CHECKSUM(nombre, segmento, fecha_alta) AS hash_fila
FROM dbo.dim_clientes;

🟢 Ejercicio: Generar lista de clientes con posible cambio (comparar hash vs valor previo simulado).

🟠 Ejercicio: Proponer estructura de tabla historial (cliente_id, campo, valor_anterior, valor_nuevo, fecha_cambio, usuario).

🔴 Reto: Simular control de versiones de precio de producto con CTE que muestre versión actual y ficticia anterior.

Errores comunes: no registrar timezone; usar métodos de hash inseguros para auditoría legal; ignorar impacto de historial en tamaño.

# Créditos

Este material fue revisado y enriquecido parcialmente mediante asistencia de IA (OpenAI y Claude); la validación y decisiones editoriales finales son humanas.

---
## Navegación
[⬅️ Anterior](07_optimizar_consultas_basico.ipynb) | [Siguiente ➡️](09_integracion_fuentes.ipynb)
---
